In [1]:
import numpy as np
import pandas as pd
import re
import collections
import operator
import simpy
import random

In [2]:
def estatistica_descritiva(data,campo):
    print(f"{campo} Media :",data[campo].mean())
    print(f"{campo} Mediana",data[campo].median())
    print(f"{campo} Moda",list(data[campo].mode()))
    print(f"{campo} Minimo",data[campo].min())
    print(f"{campo} Maximo",data[campo].max())
    print(f"{campo} Amplitude",data[campo].max()-data[campo].min())
    print(f"{campo} Desvio padrão",data[campo].std())
    print(f"{campo} Variacia",data[campo].var())
    print(f"{campo} Coeficinte de variação",data[campo].std()/data[campo].mean())
    print(f"{campo} Assimetria",data[campo].skew())

In [3]:
def tempo_medio (data,trantime=False,breaktime=False):
    
    if trantime:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos += trasacao["TranTime"]/trasacao['ArtNum']
        media_produtos_segundos = produtos_segundos/len(data.index)
        print(media_produtos_segundos)
    
    elif breaktime:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos += (trasacao["BreakTime"])/trasacao['ArtNum']
        media_produtos_segundos=produtos_segundos/len(data.index)
        print(media_produtos_segundos)
        
    else:
        produtos_segundos = 0
        for index, trasacao in data.iterrows():
            produtos_segundos +=np.add(trasacao["TranTime"],trasacao["BreakTime"])/trasacao['ArtNum']
        media_produtos_segundos=produtos_segundos/len(data.index)
        print(media_produtos_segundos)

In [4]:
trasacao = pd.read_csv("data/POS_transactions_20190328-20190410.csv")

In [5]:
trasacao.dtypes

WorkstationGroupID      int64
TranID                float64
BeginDateTime          object
EndDateTime            object
OperatorID              int64
TranTime                int64
BreakTime               int64
ArtNum                  int64
TNcash                   bool
TNcard                   bool
Amount                float64
dtype: object

In [6]:
trasacao_workstation_service = trasacao.query('WorkstationGroupID == 1')

In [7]:
trasacao_workstation_service_date = trasacao_workstation_service.loc[trasacao_workstation_service.BeginDateTime.str.contains('2019-04-03T10|2019-04-03T11|2019-04-03T12', flags=re.I, regex=True)]

## Removendo as linha com TranTime igual a zero

In [8]:
for index, transacao in trasacao_workstation_service_date.iterrows():
    if transacao["TranTime"]==0:
        trasacao_workstation_service_date=trasacao_workstation_service_date.drop([index])

## Estatistica descritiva todas as formas de pagamentos

### Estatistica descritiva campo TranTime

In [9]:
estatistica_descritiva(trasacao_workstation_service_date,"TranTime")

TranTime Media : 50.85490196078431
TranTime Mediana 41.0
TranTime Moda [26, 31]
TranTime Minimo 4
TranTime Maximo 309
TranTime Amplitude 305
TranTime Desvio padrão 40.05392393613791
TranTime Variacia 1604.3168226819214
TranTime Coeficinte de variação 0.7876118602494732
TranTime Assimetria 2.0457119278440214


### Estatistica descritiva campo Breaktime

In [10]:
estatistica_descritiva(trasacao_workstation_service_date,"BreakTime")


BreakTime Media : 33.194117647058825
BreakTime Mediana 21.0
BreakTime Moda [17]
BreakTime Minimo 6
BreakTime Maximo 1147
BreakTime Amplitude 1141
BreakTime Desvio padrão 71.33632925114235
BreakTime Variacia 5088.871871027388
BreakTime Coeficinte de variação 2.149065385910721
BreakTime Assimetria 12.041105864454947


### Estatistica Descritiva campo numeros de artigos

In [11]:
estatistica_descritiva(trasacao_workstation_service_date,"ArtNum")

ArtNum Media : 15.147058823529411
ArtNum Mediana 11.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 123
ArtNum Amplitude 122
ArtNum Desvio padrão 14.968990250918493
ArtNum Variacia 224.0706691320929
ArtNum Coeficinte de variação 0.9882440165654928
ArtNum Assimetria 2.6869050219189594


### Estatistica Descritiva campo TNcash e TNcard

In [12]:
trasacao_workstation_service_date["TNcash"].describe()

count      510
unique       2
top       True
freq       335
Name: TNcash, dtype: object

### Calculando o tempo medio de item por segundo



In [13]:
tempo_medio(trasacao_workstation_service_date)

9.205502027971132


###  Calculando o tempo medio de item por segundo considerando somente o tempo de trasação

In [14]:
tempo_medio(trasacao_workstation_service_date,trantime=True)

4.669640765279946


###  Calculando o tempo medio de item por segundo considerando somente o tempo de empacotamento

In [15]:
tempo_medio(trasacao_workstation_service_date,breaktime=True)

4.53586126269119


## Estatistica descritiva considerando apenas compras no dinheiro


In [16]:
trasacao_workstation_service_date_tncash = trasacao_workstation_service_date.query('TNcash == True')

### Estatistica descritiva campo TranTime

In [17]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"TranTime")

TranTime Media : 44.208955223880594
TranTime Mediana 35.0
TranTime Moda [11, 26, 28]
TranTime Minimo 4
TranTime Maximo 309
TranTime Amplitude 305
TranTime Desvio padrão 37.53096019896204
TranTime Variacia 1408.572973456073
TranTime Coeficinte de variação 0.8489447445410051
TranTime Assimetria 2.6301824423971367


### Estatistica descritiva campo BreakTime

In [18]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"BreakTime")

BreakTime Media : 33.56417910447761
BreakTime Mediana 22.0
BreakTime Moda [17, 19]
BreakTime Minimo 8
BreakTime Maximo 940
BreakTime Amplitude 932
BreakTime Desvio padrão 57.94443885705378
BreakTime Variacia 3357.557994458844
BreakTime Coeficinte de variação 1.7263773583344912
BreakTime Assimetria 12.235668470386164


### Estatistica descritiva campo ArtNum

In [19]:
estatistica_descritiva(trasacao_workstation_service_date_tncash,"ArtNum")

ArtNum Media : 13.170149253731344
ArtNum Mediana 10.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 123
ArtNum Amplitude 122
ArtNum Desvio padrão 13.458663871692076
ArtNum Variacia 181.13563321118957
ArtNum Coeficinte de variação 1.0219067082993756
ArtNum Assimetria 3.526173671756291


### Calculando o tempo medio de item por segundo

In [20]:
tempo_medio(trasacao_workstation_service_date_tncash)

9.980679204243227


###  Calculando o tempo medio de item por segundo considerando somente o tempo de trasação

In [21]:
tempo_medio(trasacao_workstation_service_date_tncash,trantime=True)

4.537310756315868


###  Calculando o tempo medio de item por segundo considerando somente o tempo de empacotamento

In [22]:
tempo_medio(trasacao_workstation_service_date_tncash,breaktime=True)

5.4433684479273605


## Estatistica descritiva considerando apenas no cartão de credito

In [23]:
trasacao_workstation_service_date_tncard = trasacao_workstation_service_date.query('TNcard == True')

### Estatistica descritiva campo TranTime

In [24]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"TranTime")

TranTime Media : 64.33714285714285
TranTime Mediana 52.0
TranTime Moda [31]
TranTime Minimo 13
TranTime Maximo 251
TranTime Amplitude 238
TranTime Desvio padrão 41.99487575277451
TranTime Variacia 1763.5695894909684
TranTime Coeficinte de variação 0.6527314376708002
TranTime Assimetria 1.3877977720260803


### Estatistica descritiva campo BreakTime

In [25]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"BreakTime")

BreakTime Media : 32.48
BreakTime Mediana 16.0
BreakTime Moda [14]
BreakTime Minimo 6
BreakTime Maximo 1147
BreakTime Amplitude 1141
BreakTime Desvio padrão 91.86445787511977
BreakTime Variacia 8439.078620689656
BreakTime Coeficinte de variação 2.828339220293097
BreakTime Assimetria 10.706897789257468


### Estatistica descritiva campo ArtNum

In [26]:
estatistica_descritiva(trasacao_workstation_service_date_tncard,"ArtNum")

ArtNum Media : 18.925714285714285
ArtNum Mediana 13.0
ArtNum Moda [3]
ArtNum Minimo 1
ArtNum Maximo 98
ArtNum Amplitude 97
ArtNum Desvio padrão 17.01622523306112
ArtNum Variacia 289.55192118226597
ArtNum Coeficinte de variação 0.8991061038000291
ArtNum Assimetria 1.7948384626166136


### Calculando o tempo medio de item por segundo

In [27]:
tempo_medio(trasacao_workstation_service_date_tncard)

7.877840225663656


### Calculando o tempo medio de item por transação

In [28]:
tempo_medio(trasacao_workstation_service_date_tncard,trantime=True)

5.0663667686371365


### Calculando o tempo medio item por tempo de empacotamento

In [29]:
tempo_medio(trasacao_workstation_service_date_tncard,breaktime=True)

2.8114734570265214


In [30]:
frequencia = collections.Counter(trasacao_workstation_service_date["ArtNum"])

In [31]:
frequencia_ordenada = sorted(frequencia.items(), key=operator.itemgetter(1), reverse=True)

In [32]:
frequencia_porcentagem = {}

In [33]:
frequencia_f = []
for frequencia in frequencia_ordenada:
    porcentagem = (frequencia[1]/len(trasacao_workstation_service_date.index))*100
    frequencia_f.append((frequencia[0],porcentagem))


In [34]:
frequencia_dividida_tamanho = {"qtd_pequena": [], "qtd_media": [], "qtd_grande": []}
soma_frequencia_pequena = 0
soma_frequencia_media = 0
soma_frequencia_grande = 0
for frequencia in frequencia_f:
    if frequencia[0] <= 10 :
        frequencia_dividida_tamanho["qtd_pequena"].append(frequencia[0])
        soma_frequencia_pequena += frequencia[1] 
    elif (frequencia[0] > 10) & (frequencia[0]<=30):
        frequencia_dividida_tamanho["qtd_media"].append(frequencia[0])
        soma_frequencia_media += frequencia[1]
    else :
        frequencia_dividida_tamanho["qtd_grande"].append(frequencia[0])
        soma_frequencia_grande += frequencia[1]
frequencia_dividida_tamanho["qtd_pequena"] = (soma_frequencia_pequena)
frequencia_dividida_tamanho["qtd_media"] = (soma_frequencia_media)
frequencia_dividida_tamanho["qtd_grande"] = (frequencia_dividida_tamanho["qtd_grande"],soma_frequencia_grande)

In [35]:
frequencia_dividida_tamanho["qtd_pequena"]

49.2156862745098

In [36]:
frequencia_dividida_tamanho["qtd_media"]

39.607843137254896

In [37]:
frequencia_dividida_tamanho["qtd_grande"]

([37,
  42,
  31,
  40,
  48,
  33,
  41,
  44,
  39,
  35,
  34,
  38,
  36,
  47,
  62,
  58,
  67,
  59,
  32,
  98,
  82,
  45,
  123,
  53,
  87,
  49,
  76,
  51,
  46,
  64,
  70,
  72,
  84],
 11.176470588235286)

## Calculando o tempo medio de item por segundo

In [ ]:
OperatorIDs=trasacao_workstation_service_date[['OperatorID']]

In [ ]:
operadores_unicos = OperatorIDs.drop_duplicates()
operadores_list = operadores_unicos.values

In [ ]:
operadores = {}
for operador in operadores_list:
    operadores[int(operador)] = trasacao_workstation_service_date.query(f"OperatorID=={operador}")


In [ ]:
operadores.keys()